# HuggingFace로 MNLI 문제 해결하기
허깅페이스 주소 : https://huggingface.co/soonbob/mnli-finetuned-bert-base-cased

## 라이브러리 설치

In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import random
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Dataset 준비

MNLI 데이터 다운로드 및 전처리

### 데이터셋 구성 요소
모델 학습에 필요한 '전제, 가설, 라벨' 형태로 가공한다.

premise (전제)
hypothesis (가설)
labels (라벨)

**✅ 라벨의 종류**

**함의(Entailment)**: 전제가 가설을 지 지하는 경우.​ -> 0

**중립(Neutral)**: 전제가 가설에 대해 중립적이거나 관련이 없는 경우. -> 1

**모순(Contradiction)**: 전제가 가설과 모순되는 경우.​ -> 2

In [4]:
mnli_data = load_dataset("nyu-mll/glue", "mnli")

### train 데이터 살펴보기

In [21]:
# 데이터셋의 구조 살펴보기

print(mnli_data)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [28]:
import pandas as pd
from datasets import Dataset


def display_sample(dataset: Dataset, num_samples = 5, title=""):
  sample_data = dataset.select(range(num_samples))
  df = pd.DataFrame(sample_data)
  print(f"\n{title}\n")
  display.display(df[['premise', 'hypothesis', 'label']])


display_sample(mnli_data['train'], title="🧪 MNLI train 샘플")
display_sample(mnli_data['validation_matched'], title="🧪 MNLI validation_matched 샘플")



🧪 MNLI train 샘플



,premise,hypothesis,label
0,Conceptually cream skimming has two basic dimensions - product and geography.,Product and geography are what make cream skimming work.,1
1,you know during the season and i guess at at your level uh you lose them to the next level if if...,You lose the things to the following level if the people recall.,0
2,One of our number will carry out your instructions minutely.,A member of my team will execute your orders with immense precision.,0
3,How do you know? All this is their information again.,This information belongs to them.,0
4,yeah i tell you what though if you go price some of those tennis shoes i can see why now you kno...,The tennis shoes have a range of prices.,1



🧪 MNLI validation_matched 샘플



,premise,hypothesis,label
0,The new rights are nice enough,Everyone really likes the newest benefits,1
1,This site includes a list of all award winners and a searchable database of Government Executive...,The Government Executive articles housed on the website are not able to be searched.,2
2,uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times ...,"I like him for the most part, but would still enjoy seeing someone beat him.",0
3,yeah i i think my favorite restaurant is always been the one closest you know the closest as lo...,My favorite restaurants are always at least a hundred miles away from my house.,2
4,i don't know um do you do a lot of camping,I know exactly.,2


## 데이터 전처리

In [32]:
# train 데이터를 split해서, 학습용과 validation data를 나누기

train_val_split = mnli_data['train'].train_test_split(test_size=0.2)

print(train_val_split)
train_data = train_val_split["train"]
val_data = train_val_split["test"]


DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 314161
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 78541
    })
})


**train_test_split(data, text_size, random_state)**
* 직접 train/val 데이터를 나누고 싶을 때 사용한다.
* data : 나눌 대상
* test_size=0.2 : 전체의 20%는 테스트 셋으로 나눠달라
* random_state : 랜덤 시드 고정해서 항상 같은 결과가 나오도록 함


In [40]:
import torch
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# 전처리 함수 정의
def preprocess_function(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        padding="longest",
    )


train_dataset = train_data.map(preprocess_function, batched=True)
val_dataset = val_data.map(preprocess_function, batched=True)
test_dataset = mnli_data['validation_mismatched'].map(preprocess_function, batched=True)



Map:   0%|          | 0/314161 [00:00<?, ? examples/s]

Map:   0%|          | 0/78541 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

**tokenizer()**
* BERT 계열의 토크나이저는 문장쌍까지만 구조적으로 구분 가능 =>  premise, hypothesis 문장 따로 전달
```
tokenizer(
    text = "문장1",              # 또는
    text_pair = "문장2",         # 문장쌍일 때
    truncation = True,           # 너무 길면 자름
    padding = "longest",         # 가장 긴 입력의 길이에 맞춰서 패딩 토큰을 자동으로 채워줌
    max_length = 128,            # 최대 길이
    return_tensors = "pt"        # torch 텐서로 반환 (선택)
)
```
* `[CLS] premise [SEP] hypothesis [SEP]` 구조를 사용하기로 함
* token_type_ids(문장을 구분해주는 인덱스)을 통해 모델이 문장을 쉽게 분리할 수 있도록 처리



**datasets의 map()**
* 데이터 셋에 `preprocess_function`을 적용해주는 메서드
* 배치 크기의 기본값은 1000

**데이터셋 분리**
* 학습 데이터와 validation 데이터는 train 데이터를 split해서 사용함
* 성능 테스트 데이터는 validation_matched 데이터를 사용함.



## 전처리 결과 체크 (+디코딩)

In [41]:
train_dataset[0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])

In [43]:
from transformers import AutoTokenizer
import pandas as pd
from IPython.display import display

sample = train_dataset[0]

# 디코더로 input_ids → 텍스트로 변환
# skip_special_tokens = False : [CLS], [SEP] 토큰까지 보여줌
decoded = tokenizer.decode(sample["input_ids"], skip_special_tokens=False)


df = pd.DataFrame({
    "premise": [sample["premise"]],
    "hypothesis": [sample["hypothesis"]],
    "label": [sample["label"]],
    "input_ids": [sample["input_ids"]],
    "decoded_input": [decoded]
})


display(df)


,premise,hypothesis,label,input_ids,decoded_input
0,Thanks a bunch.,I owe you one for that favor.,1,"[101, 5749, 170, 9670, 119, 102, 146, 12972, 1128, 1141, 1111, 1115, 5010, 119, 102, 0, 0, 0, 0,...",[CLS] Thanks a bunch. [SEP] I owe you one for that favor. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [P...


In [44]:
# 데이터셋 길이 체크

len(train_dataset), len(val_dataset), len(test_dataset)

(314161, 78541, 9832)

## BERT Model 구현

In [45]:
from transformers import BertConfig

config = BertConfig()

config.hidden_size = 64  # BERT layer의 기본 hidden dimension
config.intermediate_size = 64  # FFN layer의 중간 hidden dimension
config.num_hidden_layers = 2  # BERT layer의 개수
config.num_attention_heads = 4  # Multi-head attention에서 사용하는 head 개수
config.num_labels = 3  # 마지막에 예측해야 하는 분류 문제의 class 개수

model = AutoModelForSequenceClassification.from_config(config)

**AutoModelForSequenceClassification**
* 분류 문제
* config로 넘겨주는 BERT의 마지막에 linear classifier를 달아줌

## 학습 인자 정의

In [46]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='mnli-finetuned-bert-base-cased',  # 모델, log 등을 저장할 directory
    num_train_epochs=3,  # epoch 수
    per_device_train_batch_size=128,  # training data의 batch size
    per_device_eval_batch_size=128,  # validation data의 batch size
    logging_strategy="epoch",  # Epoch가 끝날 때마다 training loss 등을 log하라는 의미
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    eval_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-3,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
)

- `epochs`: training data를 몇 번 반복할 것인지
- `batch_size`: training data를 얼마나 잘게 잘라서 학습할 것인지
- `learning_rate`: optimizer의 learning rate를 얼마로 할 것인지

## 정확도 검사 정의

In [47]:
import evaluate

# accuracy 모듈을 가져옴
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")


def compute_metrics(pred):
  #Trainer가 넘겨주는 기본 포맷
    predictions, labels = pred
    #logits 중에서 가장 높은 점수를 가진 클래스를 예측값으로
    predictions = np.argmax(predictions, axis=1)
    # 정확도 계산(예측값 vs 실제값 비교)
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**eavaluate**
* HF의 evaluate 라이브러리를 활용하여 정확도 평가 지표를 쉽게 작성할 수 있다.
* accuracy 외에도 f1(정밀도와 재현율 둘 다 높은지 체크), precision(정밀도), recall(재현율) 등 다양한 지표 작성 가능


In [48]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

<ipython-input-48-2a6fcecac192>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


**Trainer 파라미터 정리**
* model : 훈련할 모델
* args : TrainingArguments 객체
* train_dataset : 학습용 데이터셋
* eval_dataset : 평가용 데이터셋
* compute_metrics : 평가할 메트릭 함수 정의 (정확도, f1 등)
* tokenizer : 토크나이저 객체 - 모델 저장/로깅 시 사용
* callbacks=[...] : 학습 중간에 특정 이벤트를 처리할 콜백함수 (early_stopping 등)

**early_stopping**
* `early_stopping_patience=2` : 평가 성능이 2번 연속 개선되지 않으면 학습 종료

학습 ㄱㄱ

In [53]:
#wandb 안씀
import os
os.environ["WANDB_DISABLED"] = "true"

import wandb
wandb.finish()  # 현재 세션 종료
wandb.init(mode="disabled") # explicitly disable wandb

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.866200,0.868188,0.603328,0.594561
2,0.796400,0.844906,0.624222,0.624192
3,0.732300,0.867255,0.623738,0.623072


TrainOutput(global_step=7365, training_loss=0.7982978111210115, metrics={'train_runtime': 1268.5338, 'train_samples_per_second': 742.97, 'train_steps_per_second': 5.806, 'total_flos': 130220993318850.0, 'train_loss': 0.7982978111210115, 'epoch': 3.0})

보시다시피 training loss는 잘 떨어지는 반면, validation loss는 중간부터 쭉 올라가는 것을 볼 수 있습니다.
Overfitting이 일어났다고 볼 수 있습니다.

위와 같이 학습이 끝난 후 validation loss가 가장 낮은 모델을 가지고 test data의 성능을 평가하는 것은 다음과 같이 구현할 수 있습니다.

In [55]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.8276292681694031,
 'eval_accuracy': 0.6367982099267697,
 'eval_f1': 0.6357611133063659,
 'eval_runtime': 5.3556,
 'eval_samples_per_second': 1835.848,
 'eval_steps_per_second': 14.378,
 'epoch': 3.0}

이전에 학습 인자에서 `load_best_model_at_end=True`를 넘겨줬기 때문에 `trainer`는 학습이 끝난 후, 기본적으로 validation loss가 가장 좋은 모델을 가지고 `evaluate`를 진행합니다.
실제로 결과를 보면 `eval_loss`가 가장 낮은 validation loss와 유사한 것을 볼 수 있습니다.

평가할 때 사용한 모델은 다음과 같이 저장할 수 있습니다.

In [56]:
trainer.save_model()

## HF에 업로드

In [58]:
from huggingface_hub import notebook_login
notebook_login()

model.safetensors:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

events.out.tfevents.1744644674.374c0244e72c.269.0:   0%|          | 0.00/17.4k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1744646662.374c0244e72c.269.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/soonbob/hf_transformer/commit/cd64111bff54a0a43361a47a0ac911a7416c8727', commit_message='mnli-finetuned-bert-base-cased', commit_description='', oid='cd64111bff54a0a43361a47a0ac911a7416c8727', pr_url=None, repo_url=RepoUrl('https://huggingface.co/soonbob/hf_transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='soonbob/hf_transformer'), pr_revision=None, pr_num=None)

In [60]:
# trainer가 저장한 모델 폴더를 push
# trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/soonbob/hf_transformer/commit/cd64111bff54a0a43361a47a0ac911a7416c8727', commit_message='soonbob/mnli-finetuned-bert-base-cased', commit_description='', oid='cd64111bff54a0a43361a47a0ac911a7416c8727', pr_url=None, repo_url=RepoUrl('https://huggingface.co/soonbob/hf_transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='soonbob/hf_transformer'), pr_revision=None, pr_num=None)

In [61]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

test_model = AutoModelForSequenceClassification.from_pretrained("soonbob/mnli-finetuned-bert-base-cased")
test_tokenizer = AutoTokenizer.from_pretrained("soonbob/mnli-finetuned-bert-base-cased")


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]